In [2]:
using TensorOperations, LinearAlgebra

In [1]:
using LinearAlgebra
using PyPlot
using RandomMatrix

In [3]:
function MPSrep_LC(psi, maxM=0)
    
    D = size(psi)[1]
    logD = log2(D)
    @assert isinteger(logD) "State vector of incompatible length"
    L = Int(logD) # How many qubits in the state?
    psiMPS = []
    VN = []
    Ms = []
    Trunc = []

    
    psi_matrix = reshape(psi, (2,2^(L-1)))
    for l in range(1, stop=L-1)
        U,s,V = svd(psi_matrix)
        Ml = findlast(s .> 1e-14) # throw out zeros
        U, s, V = U[:,1:Ml], s[1:Ml] , V[:,1:Ml]
        # first Ml columns of U corresponding to non-zero singular values
        # first Ml columns of V (i.e. Ml rows of V') corresponding to non-zero singular values
        w = s.^2
        svn = -sum(w.*log.(w))
        
        # if 0<maxM<Ml - truncation
        if(maxM > 0 && Ml > maxM) # truncate
            trunc = sum(s[maxM:end].^2)
            push!(Trunc, trunc)
            U, s, V = U[:,1:maxM], s[1:maxM] , V[:,1:maxM]
            s /= sqrt(1. - trunc)
            Ml = maxM
        end
                             
        @show Ml
        push!(Ms, Ml)
        push!(VN, svn)
        if(l == 1)
            push!(psiMPS, reshape(U, (2, Ml)))
        else
            push!(psiMPS, reshape(U, (Ms[end-1], 2, Ml)))
        end
        if(l == L-1)
            push!(psiMPS, reshape(diagm(s)*V', (Ml, 2)))    
        else
            psi_matrix = reshape(diagm(s)*V', (2*Ml,2^(L-l-1)))    
        end
    end
    return psiMPS, VN, Ms, Trunc
end

MPSrep_LC (generic function with 2 methods)

## Cat State

In [4]:
L = 3
D = 2^L
psi = zeros(D)
psi[1] = 1.  #cat state
psi[end] = 1. #cat state
psi ./= norm(psi)
psi
psiMPS, VN, Ms, Trunc = MPSrep_LC(psi)
VN

Ml = 2
Ml = 2


2-element Vector{Any}:
 0.6931471805599454
 0.6931471805599454

### up spins

In [5]:
psiMPS[1][1,:] # spin up

2-element Vector{Float64}:
 1.0
 0.0

In [6]:
psiMPS[2][:,1,:] # spin up

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  0.0

In [7]:
psiMPS[3][:,1,:] # spin up

2×1 Matrix{Float64}:
 0.7071067811865475
 0.0

In [ ]:
psiMPS[4][:,1] # spin up

### down spins

In [8]:
psiMPS[1][2,:] # spin down

2-element Vector{Float64}:
 0.0
 1.0

In [9]:
psiMPS[2][:,2,:] # spin down

2×2 Matrix{Float64}:
 0.0   0.0
 0.0  -1.0

In [10]:
psiMPS[3][:,2,:] # spin down

2×1 Matrix{Float64}:
  0.0
 -0.7071067811865475

In [ ]:
psiMPS[4][:,2] # spin down

the minus sign is exactly the gauge freedom shown in class.

## Random state

In [12]:
L = 10
D = 2^L
psi = rand(D)
psi ./= norm(psi)
psi
println("Bond dims - no truncation:")
psiMPS, VN, Ms, Trunc = MPSrep_LC(psi);
println("Bond dims - with truncation:")
psiMPS1, VN1, Ms1, Trunc1 = MPSrep_LC(psi, 10);
# VN


Bond dims - no truncation:
Ml = 2
Ml = 4
Ml = 8
Ml = 16
Ml = 32
Ml = 16
Ml = 8
Ml = 4
Ml = 2
Bond dims - with truncation:
Ml = 2
Ml = 4
Ml = 8
Ml = 10
Ml = 10
Ml = 10
Ml = 8
Ml = 4
Ml = 2


In [ ]:
plot(VN)
# plot(log.(Ms))

## MPS - tensor contraction

In [ ]:
psiMPS[3]

In [13]:
L = size(psiMPS)[1]

M = size(psiMPS[2])[3]
psi_bond = zeros(2,2,M)
@tensor psi_bond[s1, s2, l2] = psiMPS[1][s1, l1]*psiMPS[2][l1, s2, l2]
psi_bond = reshape(psi_bond,(4,M))

for b in range(2, stop=L-2)
    println("loop")
    M = size(psiMPS[b+1])[3]
    psi_new = zeros(2^b,2,M)
    @tensor psi_new[s1, s2, l2] = psi_bond[s1, l1]*psiMPS[b+1][l1, s2, l2]
    psi_bond = reshape(psi_new,(2^(b+1),M))
end

psi_new = zeros(2^(L-1),2)
@tensor psi_new[s1, s2] = psi_bond[s1, l1]*psiMPS[end][l1, s2]
psi_bond = reshape(psi_new,(2^L,1))



loop
loop
loop
loop
loop
loop
loop


1024×1 Matrix{Float64}:
 0.004476193853985417
 0.026393202361319902
 0.047170227690590816
 0.0018399350371513627
 0.0011289514856492322
 0.04469086683581759
 0.053880621650542734
 0.047097220232489426
 0.04735887650276042
 0.015102271599243898
 0.04327266720350975
 0.0034768784040758284
 0.04959083473107147
 ⋮
 0.010251318624936594
 0.05131866935936781
 0.029583579206680997
 0.01932465903596973
 0.05163116495862409
 0.04541115089764966
 0.010626125393752349
 0.0032249328004816044
 0.047914371331524624
 0.017735407770524743
 0.03314261088032239
 0.01630928502388222

In [14]:
dot(psi, psi_bond)

1.0000000000000007